# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500)

]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (3, User(id=3, age=33, social_contacts=800, gender='female')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(33, User(id=3, age=33, social_contacts=800, gender='female')),
 (25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(33, [User(id=3, age=33, social_contacts=800, gender='female')]),
 (25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(33, 800.0), (25, 370.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(33, 800.0), (25, 370.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates, value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.0303202422407),
 (1, 2.0303202422407),
 (2, 2.0303202422407),
 (3, 2.0303202422407),
 (4, 2.0303202422407)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREADER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [('', 6)]),
 (1, [('a', 2), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.10553294889532361),
   (None, 0.1191956634799749),
   (None, 0.26788156447952816),
   (None, 0.32549378262476736),
   (None, 0.34053611645670145),
   (None, 0.3417256473460688),
   (None, 0.34178876726943674),
   (None, 0.4005064523626628),
   (None, 0.40556603976362837),
   (None, 0.41039544884152357),
   (None, 0.43865609012759077),
   (None, 0.44235334181597885),
   (None, 0.47834132290770937),
   (None, 0.48846642601367496)]),
 (1,
  [(None, 0.5859748594616956),
   (None, 0.61605692703846),
   (None, 0.6178219103897659),
   (None, 0.6318135002358314),
   (None, 0.6573014593096161),
   (None, 0.7020332103047535),
   (None, 0.7401052589181255),
   (None, 0.7517657341033622),
   (None, 0.7712807381991232),
   (None, 0.7747785947950669),
   (None, 0.8851651300010539),
   (None, 0.8987474038735705),
   (None, 0.9291603019551774),
   (None, 0.9470463889918722),
   (None, 0.9624543507913116),
   (None, 0.9722007545747855)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
vect = np.random.rand(10)

def MAP(_, value):
  yield (None, value)

def REDUCE(_, val):
  yield max(val)

def RECORDREADER():
  for i in range(len(vect)):
    yield(None, vect[i])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[0.9977453256798631]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
vect = np.random.rand(10)

def MAP(_, value):
  yield (None, value)

def REDUCE(_, val):
  yield np.mean(val)

def RECORDREADER():
  for i in range(len(vect)):
    yield(None, vect[i])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[0.5190838231755236]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return sorted(t.items(), key=lambda x: x[0])

print(list(groupbykey(map_output)))

[(25, [User(id=1, age=25, social_contacts=240, gender='female'), User(id=2, age=25, social_contacts=500, gender='female')]), (33, [User(id=3, age=33, social_contacts=800, gender='female')])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
col = [
    User(id=0, age=54, gender="male", social_contacts=20),
    User(id=3, age=25, gender="female", social_contacts=240),
    User(id=1, age=27, gender="male", social_contacts=642),
    User(id=2, age=16, gender="male", social_contacts=123),
    User(id=1, age=35, gender="female", social_contacts=247),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=3, age=32, gender="female", social_contacts=753),
]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  def RECORDREADER(split):
    for user in split:
      yield (None, user)

  split_size =  int(np.ceil(len(col)/maps))
  for i in range(0, len(col), split_size):
    yield RECORDREADER(col[i:i+split_size])

def MAP(_, user:NamedTuple):
  yield user.id, user

def REDUCE(userId:int, user:Iterator[str]):
  yield user[0]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

7 key-value pairs were sent over a network.


[(0, [User(id=0, age=54, social_contacts=20, gender='male')]),
 (1,
  [User(id=1, age=27, social_contacts=642, gender='male'),
   User(id=2, age=16, social_contacts=123, gender='male'),
   User(id=3, age=25, social_contacts=240, gender='female')])]

#Операторы реляционной алгебры

### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
class Customer(NamedTuple):
  id: int
  name: str
  phone: int
  pet_id: int
  pet_name: str
  pet_type: str
  pet_age: int

customers = [
    Customer(id=1, name="Anna Ivanova", phone="88000000001", pet_id=2, pet_name="Murka", pet_type="cat", pet_age=13),
    Customer(id=2, name="Elena Petrova", phone="88000000002", pet_id=3, pet_name="Barsik", pet_type="dog", pet_age=15),
    Customer(id=3, name="Maria Lol", phone="88035667267", pet_id=1, pet_name="Boy", pet_type="dog", pet_age=10)
]
customers2 = [
    Customer(id=4, name="Artem Ivanov", phone="83583746001", pet_id=2, pet_name="Melissa", pet_type="cat", pet_age=2),
    Customer(id=5, name="Dasha Arkhipova", phone="83465782092", pet_id=3, pet_name="Marusya", pet_type="cat", pet_age=6),
    Customer(id=2, name="Elena Petrova", phone="88000000002", pet_id=3, pet_name="Barsik", pet_type="dog", pet_age=15),
    Customer(id=6, name="Nikita Kozlov", phone="88473469392", pet_id=1, pet_name="Baby", pet_type="man", pet_age=21)
]

In [26]:
def MAP(_, row:NamedTuple):
  if (row.pet_type == 'dog'):
    yield (row, row)

def REDUCE(row1:NamedTuple, row2:NamedTuple):
  yield (row1, row2)

def RECORDREADER():
  return [(u.id, u) for u in customers]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Customer(id=2, name='Elena Petrova', phone='88000000002', pet_id=3, pet_name='Barsik', pet_type='dog', pet_age=15),
  [Customer(id=2, name='Elena Petrova', phone='88000000002', pet_id=3, pet_name='Barsik', pet_type='dog', pet_age=15)]),
 (Customer(id=3, name='Maria Lol', phone='88035667267', pet_id=1, pet_name='Boy', pet_type='dog', pet_age=10),
  [Customer(id=3, name='Maria Lol', phone='88035667267', pet_id=1, pet_name='Boy', pet_type='dog', pet_age=10)])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [27]:
def MAP(_, row:NamedTuple):
  yield (row.pet_type, row.pet_type)

def REDUCE(row1:NamedTuple, row2:NamedTuple):
  yield (row1, row2[0])

def RECORDREADER():
  return [(u.id, u) for u in customers]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('cat', 'cat'), ('dog', 'dog')]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [28]:
def MAP(_, row:NamedTuple):
  yield (row, row)

def REDUCE(row1:NamedTuple, row2:NamedTuple):
  yield (row1, row2[0])

def RECORDREADER():
  return [(u.id, u) for u in customers+customers2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Customer(id=1, name='Anna Ivanova', phone='88000000001', pet_id=2, pet_name='Murka', pet_type='cat', pet_age=13),
  Customer(id=1, name='Anna Ivanova', phone='88000000001', pet_id=2, pet_name='Murka', pet_type='cat', pet_age=13)),
 (Customer(id=2, name='Elena Petrova', phone='88000000002', pet_id=3, pet_name='Barsik', pet_type='dog', pet_age=15),
  Customer(id=2, name='Elena Petrova', phone='88000000002', pet_id=3, pet_name='Barsik', pet_type='dog', pet_age=15)),
 (Customer(id=3, name='Maria Lol', phone='88035667267', pet_id=1, pet_name='Boy', pet_type='dog', pet_age=10),
  Customer(id=3, name='Maria Lol', phone='88035667267', pet_id=1, pet_name='Boy', pet_type='dog', pet_age=10)),
 (Customer(id=4, name='Artem Ivanov', phone='83583746001', pet_id=2, pet_name='Melissa', pet_type='cat', pet_age=2),
  Customer(id=4, name='Artem Ivanov', phone='83583746001', pet_id=2, pet_name='Melissa', pet_type='cat', pet_age=2)),
 (Customer(id=5, name='Dasha Arkhipova', phone='83465782092', pet_id=3, 

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [29]:
def MAP(_, row:NamedTuple):
  yield (row, row)

def REDUCE(row1:NamedTuple, row2:NamedTuple):
  if (len(row2) == 2):
    yield (row1, row2[0])

def RECORDREADER():
  return [(u.id, u) for u in customers+customers2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Customer(id=2, name='Elena Petrova', phone='88000000002', pet_id=3, pet_name='Barsik', pet_type='dog', pet_age=15),
  Customer(id=2, name='Elena Petrova', phone='88000000002', pet_id=3, pet_name='Barsik', pet_type='dog', pet_age=15))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [30]:
def MAP(_, row:NamedTuple):
  if (row in customers2):
    yield (row, 'S')
  else:
    yield (row, 'R')

def REDUCE(row:NamedTuple, bit):
  if (bit[0] == 'R'):
    yield (row, row)

def RECORDREADER():
  return [(u.id, u) for u in customers+customers2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Customer(id=1, name='Anna Ivanova', phone='88000000001', pet_id=2, pet_name='Murka', pet_type='cat', pet_age=13),
  Customer(id=1, name='Anna Ivanova', phone='88000000001', pet_id=2, pet_name='Murka', pet_type='cat', pet_age=13)),
 (Customer(id=3, name='Maria Lol', phone='88035667267', pet_id=1, pet_name='Boy', pet_type='dog', pet_age=10),
  Customer(id=3, name='Maria Lol', phone='88035667267', pet_id=1, pet_name='Boy', pet_type='dog', pet_age=10))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [31]:
class Doct(NamedTuple):
  id: int
  name: str
  pet_id: int

doctors = [
    Doct(id=7, name="Arina P", pet_id=2),
    Doct(id=8, name="Kristina D", pet_id=3),
    Doct(id=9, name="Andrey S", pet_id=1),
    Doct(id=9, name="Polina T", pet_id=10)
]

In [32]:
def MAP(_, row:NamedTuple):
  if ((row in customers) or (row in customers2)):
      yield (row.pet_id, ('R', (row.id, row.name, row.phone, row.pet_name, row.pet_type, row.pet_age)))
  elif (row in doctors) and row.pet_id in [row1.pet_id for row1 in customers+customers2]:
    yield (row.pet_id, ('S', (row.id, row.name)))

def REDUCE(key, info:NamedTuple):
  for rowS in info:
    if (rowS[0] == 'S'):
      for rowR in info:
        if (rowR[0] == 'R'):
          yield (None, (rowR[1], key, rowS[1]))

def RECORDREADER():
  return [(u.id, u) for u in customers+customers2+doctors]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(None,
  ((3, 'Maria Lol', '88035667267', 'Boy', 'dog', 10), 1, (9, 'Andrey S'))),
 (None,
  ((6, 'Nikita Kozlov', '88473469392', 'Baby', 'man', 21),
   1,
   (9, 'Andrey S'))),
 (None,
  ((1, 'Anna Ivanova', '88000000001', 'Murka', 'cat', 13), 2, (7, 'Arina P'))),
 (None,
  ((4, 'Artem Ivanov', '83583746001', 'Melissa', 'cat', 2),
   2,
   (7, 'Arina P'))),
 (None,
  ((2, 'Elena Petrova', '88000000002', 'Barsik', 'dog', 15),
   3,
   (8, 'Kristina D'))),
 (None,
  ((5, 'Dasha Arkhipova', '83465782092', 'Marusya', 'cat', 6),
   3,
   (8, 'Kristina D'))),
 (None,
  ((2, 'Elena Petrova', '88000000002', 'Barsik', 'dog', 15),
   3,
   (8, 'Kristina D')))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [33]:
def MAP(_, row:NamedTuple):
  yield (row.pet_type, row.pet_age)

def REDUCE(key, row):
  yield(key, max(row))

def RECORDREADER():
  return [(u.id, u) for u in customers+customers2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('cat', 13), ('dog', 15), ('man', 21)]

#Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи

In [34]:
from typing import Iterator
import numpy as np

d = 4
mat = np.ones((d+1,d))
vec1 = np.random.rand(d//2) # in-memory vector in all map tasks
vec2 = np.random.rand(d-d//2) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  if (j < len(vec1)):
    yield (i, value*vec1[j])
  else:
    yield (i, value*vec2[j-len(vec1)])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.8813375964709018),
 (1, 1.8813375964709018),
 (2, 1.8813375964709018),
 (3, 1.8813375964709018),
 (4, 1.8813375964709018)]

#Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [35]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [36]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    v2 = (small_mat[i][j]*w)
    k2 = (i, k)
    yield (k2, v2)
  # solution code that yield(k2,v2) pairs

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))
  # solution code that yield(k3,v3) pairs

Проверьте своё решение

In [37]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [38]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [39]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield (((i,j), small_mat[i,j]),((j,k), big_mat[j,k]))

def MAP(list1, list2):
  (k1, v1) = list1
  (k2, v2) = list2
  yield ((k1[0], k2[1]), v1*v2)

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

In [40]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [41]:
import numpy as np

I = 2
J = 3
K = 4*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
  def RECORDREADER(key, split):
    mat = []
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        mat.append(((key, i, j), split[i, j]))
    return mat

  list1 = RECORDREADER('S', small_mat)
  list2 = RECORDREADER('B', big_mat)
  yield list1
  yield list2


def MAP(k, v):
  (mat, i, j) = k
  w = v
  if mat == 'S':
    yield (j, (mat, i, w))
  else:
    yield (i, (mat, j, w))

def REDUCE(key, values):
  small = [v for v in values if v[0] == 'S']
  big = [v for v in values if v[0] == 'B']
  for s in small:
    for b in big:
      yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


In [42]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [43]:
import numpy as np

I = 2
J = 3
K = 4*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
  global maps
  def RECORDREADER(key, split):
    mat = []
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        mat.append(((key, i, j), split[i, j]))
    return mat

  small = RECORDREADER('S', small_mat)
  split_size =  int(np.ceil(len(small)/maps))
  for i in range(0, len(small), split_size):
    yield small[i:i+split_size]

  big = RECORDREADER('B', big_mat)
  split_size =  int(np.ceil(len(big)/maps))
  for i in range(0, len(big), split_size):
    yield big[i:i+split_size]

def MAP(k, v):
  (mat, i, j) = k
  w = v
  if mat == 'S':
    yield (j, (mat, i, w))
  else:
    yield (i, (mat, j, w))

def REDUCE(key, values):
  small = [v for v in values if v[0] == 'S']
  big = [v for v in values if v[0] == 'B']
  for s in small:
    for b in big:
      yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


In [44]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True